In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import string
import warnings
from pathlib import Path
from time import time

import arviz as az
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotnine as gg
import pymc3 as pm
import seaborn as sns
from theano import tensor as tt

In [3]:
from src.data_processing import achilles as achelp
from src.data_processing import common as dphelp
from src.modeling import pymc3_analysis as pmanal
from src.modeling import pymc3_sampling_api as pmapi
from src.models.speclet_three import SpecletThree
from src.plot.color_pal import ModelColors, SeabornColor

In [4]:
notebook_tic = time()

warnings.simplefilter(action="ignore", category=UserWarning)

gg.theme_set(
    gg.theme_bw()
    + gg.theme(
        figure_size=(4, 4),
        axis_ticks_major=gg.element_blank(),
        strip_background=gg.element_blank(),
    )
)
%config InlineBackend.figure_format = "retina"

RANDOM_SEED = 847
np.random.seed(RANDOM_SEED)

In [5]:
cache_dir = Path("temp", "speclet-three-debugging")
sp = SpecletThree(
    "DEBUGGING",
    root_cache_dir=cache_dir,
    debug=True,
    kras_cov=True,
    kras_mutation_minimum=3,
)
print(sp)

Speclet Model: 'speclet-two_DEBUGGING' (debug)


In [6]:
data = sp.data_manager.get_data()

[04/30/21 14:52:26] WARNING  Dropping data points of sgRNA that    ]8;id=1619808746.9950771-453546;file:///Users/admin/Lab_Projects/speclet/src/managers/model_data_managers.py\model_data_managers.py]8;;\:125
                             map to multiple genes.

[04/30/21 14:52:27] WARNING  Dropping data points with missing     ]8;id=1619808747.0550091-912794;file:///Users/admin/Lab_Projects/speclet/src/managers/model_data_managers.py\model_data_managers.py]8;;\:131
                             copy number.

In [7]:
sp.build_model()

                    INFO     Calling `model_specification()` method.     ]8;id=1619808747.284609-741950;file:///Users/admin/Lab_Projects/speclet/src/models/speclet_model.py\speclet_model.py]8;;\:161

                    INFO     Beginning PyMC3 model specification.        ]8;id=1619808747.2881558-484794;file:///Users/admin/Lab_Projects/speclet/src/models/speclet_three.py\speclet_three.py]8;;\:101

                    INFO     Getting Theano shared variables.            ]8;id=1619808747.393882-514581;file:///Users/admin/Lab_Projects/speclet/src/models/speclet_three.py\speclet_three.py]8;;\:108

                    INFO     Creating PyMC3 model.                       ]8;id=1619808747.406165-895405;file:///Users/admin/Lab_Projects/speclet/src/models/speclet_three.py\speclet_three.py]8;;\:116

In [8]:
print(sp.model)

      μ_h ~ Normal
σ_h_log__ ~ TransformedDistribution
        h ~ Normal
      μ_g ~ Normal
σ_g_log__ ~ TransformedDistribution
        g ~ Normal
      μ_b ~ Normal
σ_b_log__ ~ TransformedDistribution
        b ~ Normal
      μ_a ~ Normal
σ_a_log__ ~ TransformedDistribution
        a ~ Normal
σ_σ_log__ ~ TransformedDistribution
  σ_log__ ~ TransformedDistribution
      σ_h ~ HalfNormal
      σ_g ~ HalfNormal
      σ_b ~ HalfNormal
      σ_a ~ HalfNormal
        μ ~ Deterministic
      σ_σ ~ HalfNormal
        σ ~ HalfNormal
      lfc ~ Normal


In [9]:
sp.model["a"].dshape

(99, 5)

In [10]:
dphelp.nunique(data["hugo_symbol"]), dphelp.nunique(data["kras_mutation"])

(99, 10)

In [ ]:
sp.advi_sample_model(n_iterations=100)

/usr/local/Caskroom/miniconda/base/envs/speclet/lib/python3.9/site-packages/pymc3/data.py:316: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/Caskroom/miniconda/base/envs/speclet/lib/python3.9/site-packages/pymc3/data.py:316: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


[04/30/21 14:53:00] INFO     Beginning ADVI fitting.                     ]8;id=1619808780.206561-322441;file:///Users/admin/Lab_Projects/speclet/src/models/speclet_model.py\speclet_model.py]8;;\:387

Finished [100%]: Average Loss = 28,031


---

In [ ]:
notebook_toc = time()
print(f"execution time: {(notebook_toc - notebook_tic) / 60:.2f} minutes")

In [ ]:
%load_ext watermark
%watermark -d -u -v -iv -b -h -m